In [1]:
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from glob import glob
import torchvision
import torchvision.transforms as transforms

import pickle
import sys
import os

from opacus.validators import ModuleValidator
from opacus.accountants.utils import get_noise_multiplier
from private_vision import PrivacyEngine
import opacus
from tqdm import tqdm
import warnings
import timm
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import json

In [2]:
data_dir = '/s/lovelace/c/nobackup/iray/dp-imgclass/HAM10000_original'
print(os.listdir(data_dir))

['hmnist_8_8_RGB.csv', 'hmnist_28_28_L.csv', 'hmnist_28_28_RGB.csv', 'HAM10000_images_part_2', 'HAM10000_images_part_1', 'HAM10000_metadata.csv', 'hmnist_8_8_L.csv']


In [3]:
all_image_path = glob(os.path.join(data_dir, '*', '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}


In [4]:
df_original = pd.read_csv(os.path.join(data_dir, 'HAM10000_metadata.csv'))
df_original['path'] = df_original['image_id'].map(imageid_path_dict.get)
df_original['cell_type'] = df_original['dx'].map(lesion_type_dict.get)
df_original['cell_type_idx'] = pd.Categorical(df_original['cell_type']).codes
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2


In [5]:
df_undup = df_original.groupby('lesion_id').count()
# now we filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image_id'] == 1]
df_undup.reset_index(inplace=True)
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

# create a new colum that is a copy of the lesion_id column
df_original['duplicates'] = df_original['lesion_id']
# apply the function to this new column
df_original['duplicates'] = df_original['duplicates'].apply(get_duplicates)
df_original.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx,duplicates
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2,duplicated
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2,duplicated
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2,duplicated
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2,duplicated
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/s/lovelace/c/nobackup/iray/dp-imgclass/HAM100...,Benign keratosis-like lesions,2,duplicated


In [6]:
df_undup = df_original[df_original['duplicates'] == 'unduplicated']
df_undup.shape

(5514, 11)

In [38]:
y = df_undup['cell_type_idx']
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=101, stratify=y)
df_val.shape, df_val['cell_type_idx'].value_counts()


((1103, 11),
 cell_type_idx
 4    883
 2     88
 5     46
 1     35
 0     30
 6     13
 3      8
 Name: count, dtype: int64)

In [32]:
# This set will be df_original excluding all rows that are in the val set
# This function identifies if an image is part of the train or val set.
def get_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image_id'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# identify train and val rows
# create a new colum that is a copy of the image_id column
df_original['train_or_val'] = df_original['image_id']
# apply the function to this new column
df_original['train_or_val'] = df_original['train_or_val'].apply(get_val_rows)
# filter out train rows
df_train = df_original[df_original['train_or_val'] == 'train']
print(len(df_train))
print(len(df_val))

8912
1103


In [21]:
df_train['cell_type'].value_counts()

cell_type
Melanocytic nevi                  5822
Melanoma                          1067
Benign keratosis-like lesions     1011
Basal cell carcinoma               479
Actinic keratoses                  297
Vascular lesions                   129
Dermatofibroma                     107
Name: count, dtype: int64

In [28]:
df_train['cell_type_idx'].value_counts()

cell_type_idx
4    5822
5    1067
2    1011
1     479
0     297
6     129
3     107
Name: count, dtype: int64

In [26]:
df_val['cell_type'].value_counts()

cell_type
Melanocytic nevi                  883
Benign keratosis-like lesions      88
Melanoma                           46
Basal cell carcinoma               35
Actinic keratoses                  30
Vascular lesions                   13
Dermatofibroma                      8
Name: count, dtype: int64

In [27]:
df_val['cell_type_idx'].value_counts()

cell_type_idx
4    883
2     88
5     46
1     35
0     30
6     13
3      8
Name: count, dtype: int64

In [40]:
df_val['cell_type'].value_counts()

cell_type
Melanocytic nevi                  883
Benign keratosis-like lesions     880
Basal cell carcinoma              875
Vascular lesions                  871
Actinic keratoses                 840
Melanoma                          828
Dermatofibroma                    800
Name: count, dtype: int64

In [33]:
traindata_aug_rate = [15,10,5,50,0,5,40]
for i in range(7):
    if traindata_aug_rate[i]:
        aug_list = [df_train.loc[df_train['cell_type_idx'] == i,:]]*(traindata_aug_rate[i]-1)
        aug_df = pd.concat(aug_list)
        df_train = pd.concat([df_train, aug_df])

df_train = df_train.reset_index()


In [39]:
testdata_aug_rate = [28,25,10,100,0,18,67]
for i in range(7):
    if testdata_aug_rate[i]:
        aug_list = [df_val.loc[df_val['cell_type_idx'] == i,:]]*(testdata_aug_rate[i]-1)
        aug_df = pd.concat(aug_list)
        df_val = pd.concat([df_val, aug_df])

df_val = df_val.reset_index()

In [41]:
train_transform = transforms.Compose([transforms.Resize((224,224)),transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                        transforms.ToTensor()])

class HAM10000(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

trainset = HAM10000(df_train, transform=train_transform)
# Same for the validation set:
testset = HAM10000(df_val, transform=train_transform)


In [42]:
def prepare(args):
    print('==> Building model..', args.model, '  mode ', args.mode)
    device=torch.device("cuda")
    NUM_CLASSES=7
    trainloader = DataLoader(trainset, batch_size=args.mini_bs, shuffle=True, num_workers=4)
    testloader = DataLoader(testset, batch_size=args.mini_bs, shuffle=False, num_workers=4)

    net = timm.create_model(args.model, pretrained=args.pretrained, num_classes=NUM_CLASSES)

    # 
    net = ModuleValidator.fix(net)
    net = net.to(device)

    if 'xcit' in args.model:
      for name,param in net.named_parameters():
          if 'gamma' in name or 'attn.temperature' in name:
            param.requires_grad=False
 
    if 'cait' in args.model:
      for name,param in net.named_parameters():
          if 'gamma_' in name:
            param.requires_grad=False

    if 'convnext' in args.model:
        for name,param in net.named_parameters():
            if '.gamma' in name or 'head.norm.' in name or 'downsample.0' in name or 'stem.1' in name:
                param.requires_grad=False

    if 'convit' in args.model:
        for name,param in net.named_parameters():
            if 'attn.gating_param' in name:
                param.requires_grad=False

    if 'beit' in args.model:
        for name,param in net.named_parameters():
            if 'gamma_' in name or 'relative_position_bias_table' in name or 'attn.qkv.weight' in name or 'attn.q_bias' in name or 'attn.v_bias' in name:
                param.requires_grad=False

    for name,param in net.named_parameters():
        if 'cls_token' in name or 'pos_embed' in name:
            param.requires_grad=False

    print('number of parameters: ', sum([p.numel() for p in net.parameters()]))

    if "ghost" in args.mode:
        criterion = nn.CrossEntropyLoss(reduction="none")
    else:
        criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(net.parameters(), lr=args.lr)

    n_acc_steps = args.bs // args.mini_bs

    if 'ghost' in args.mode:
        sigma = get_noise_multiplier(
                target_epsilon = args.eps,
                target_delta = 1e-5,
                sample_rate = args.bs/len(trainset),
                epochs = args.epochs,
                accountant = "gdp"
            )
        privacy_engine = PrivacyEngine(
            net,
            batch_size=args.bs,
            sample_size=len(trainloader.dataset),
            noise_multiplier=sigma,
            epochs=args.epochs,
            max_grad_norm=args.grad_norm,
            ghost_clipping='non' not in args.mode,
            mixed='mixed' in args.mode
        )
        privacy_engine.attach(optimizer)

    def train(epoch):
        print('\nEpoch: %d' % epoch)
        net.train()
        train_loss = 0
        correct = 0
        total = 0

        for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            if args.mode=='non-private':
                loss.backward()
                if ((batch_idx + 1) % n_acc_steps == 0) or ((batch_idx + 1) == len(trainloader)):
                    optimizer.step()
                    optimizer.zero_grad()
            else:
                if ((batch_idx + 1) % n_acc_steps == 0) or ((batch_idx + 1) == len(trainloader)):
                    optimizer.step(loss=loss)
                    optimizer.zero_grad()
                else:
                    optimizer.virtual_step(loss=loss)
            train_loss += loss.mean().item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print(epoch, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        return train_loss/(batch_idx+1), 100.*correct/total

    
    def test(epoch):
        net.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(tqdm(testloader)):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                loss = loss.mean()
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

            print(epoch, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            return test_loss/(batch_idx+1), 100.*correct/total
    def save_model(filepath):
        with open(filepath, 'wb') as f:
            pickle.dump(net, f)
            # torch.save(net,f)

    return args.epochs, train, test, save_model

In [43]:
def main(epochs, trainf, testf, args):
    for epoch in range(epochs):
        train_loss, train_accuracy = trainf(epoch)
        test_loss, test_accuracy = testf(epoch)
    return train_loss, train_accuracy, test_loss, test_accuracy

In [48]:
# block for Non private training

class Arguments:
    def __init__(self, learning_rate, epochs, batch_size, epsilon, grad_norm, mode, model, mini_batch_size, pretrained):
        self.lr = learning_rate
        self.epochs = epochs
        self.bs = batch_size
        self.eps = epsilon
        self.grad_norm = grad_norm
        self.mode = mode
        self.model = model
        self.mini_bs = mini_batch_size
        self.pretrained = pretrained
    def __str__(self):
        return f"Arguments(learning_rate={self.lr}, epochs={self.epochs}, batch_size={self.bs}, epsilon={self.eps}, grad_norm={self.grad_norm}, mode='{self.mode}', model='{self.model}', mini_batch_size={self.mini_bs}, pretrained={self.pretrained})"

results = [['Epsilon', 'Grad Norm', 'Mini Batch Size', 'Train Loss', 'Train Accuracy', 'Test Loss', 'Test Accuracy']]
epsilons = [0.5, 1, 2]
batch_sizes = [800, 1600]
epochs_to_test = [5, 10]
mini_batches = [10,20]
learning_rates = []

# batch_sizes = [500]
# epochs_to_test = [5]

# for batch_size in batch_sizes:
#     for epoch in epochs_to_test:
#         # Modify any additional variables here (or put them into their own loop like above)
#         args = Arguments(learning_rate=1e-4,
#                         epochs=epoch,
#                         batch_size=batch_size,
#                         epsilon=1,
#                         grad_norm=0.5,
#                         mode='non-private',
#                         model='beit_base_patch16_224.in22k_ft_in22k',
#                         mini_batch_size=10,
#                         pretrained=1)
#         epochs, trainf, testf, save_modelf = prepare(args)
#         print(f"Running experiment with: {args}")
#         train_loss, train_accuracy, test_loss, test_accuracy = main(epochs, trainf, testf, args)
#         results.append([batch_size, epoch, train_loss, train_accuracy, test_loss, test_accuracy])
#     with open(f"ham_results/results_non-private_{args.bs}_{args.eps}_{args.mini_bs}.json", 'w') as result_file:
#             result_file.write(json.dumps(results))

# for batch_size in batch_sizes:
#     for epsilon in epsilons:
#         for mini_bs in mini_batches:
#             for epoch in epochs_to_test:
#                 # Modify any additional variables here (or put them into their own loop like above)
#                 args = Arguments(learning_rate=1e-3,
#                                 epochs=epoch,
#                                 batch_size=batch_size,
#                                 epsilon=epsilon,
#                                 grad_norm=0.5,
#                                 mode='ghost_mixed',
#                                 model='beit_base_patch16_224.in22k_ft_in22k',
#                                 mini_batch_size=mini_bs,
#                                 pretrained=0)
#                 epochs, trainf, testf, save_modelf = prepare(args)
#                 print(f"Running experiment with: {args}")
#                 train_loss, train_accuracy, test_loss, test_accuracy = main(epochs, trainf, testf, args)
#                 results.append([epsilon, 0.5, mini_bs, train_loss, train_accuracy, test_loss, test_accuracy])
#             with open(f"ham_results/results_no-pretrained_{args.bs}_{args.eps}_{args.mini_bs}.json", 'w') as result_file:
#                     result_file.write(json.dumps(results))

args = Arguments(learning_rate=1e-3,
                epochs=5,
                batch_size=1600,
                epsilon=2,
                grad_norm=0.5,
                mode='non-ghost',
                model='beit_base_patch16_224.in22k_ft_in22k',
                mini_batch_size=10,
                pretrained=1)
epochs, trainf, testf, save_modelf = prepare(args)
print(f"Running experiment with: {args}")
train_loss, train_accuracy, test_loss, test_accuracy = main(epochs, trainf, testf, args)

==> Building model.. beit_base_patch16_224.in22k_ft_in22k   mode  non-ghost
number of parameters:  85767367
Running experiment with: Arguments(learning_rate=0.001, epochs=5, batch_size=1600, epsilon=2, grad_norm=0.5, mode='non-ghost', model='beit_base_patch16_224.in22k_ft_in22k', mini_batch_size=10, pretrained=1)

Epoch: 0


  0%|          | 0/3597 [00:00<?, ?it/s]


AttributeError: 'Parameter' object has no attribute 'grad_sample'
 *** patch_embed.proj.weight parameter has no grad_sample attribute ***

In [18]:
class Arguments:
    def __init__(self, learning_rate, epochs, batch_size, epsilon, grad_norm, mode, model, mini_batch_size, pretrained):
        self.lr = learning_rate
        self.epochs = epochs
        self.bs = batch_size
        self.eps = epsilon
        self.grad_norm = grad_norm
        self.mode = mode
        self.model = model
        self.mini_bs = mini_batch_size
        self.pretrained = pretrained
    def __str__(self):
        return f"Arguments(learning_rate={self.lr}, epochs={self.epochs}, batch_size={self.bs}, epsilon={self.eps}, grad_norm={self.grad_norm}, mode='{self.mode}', model='{self.model}', mini_batch_size={self.mini_bs}, pretrained={self.pretrained})"

results = [['Epsilon', 'Grad Norm', 'Mini Batch Size', 'Train Loss', 'Train Accuracy', 'Test Loss', 'Test Accuracy']]
epsilons = [0.5, 1, 2]
batch_sizes = [800, 1600]
epochs_to_test = [5, 10]
mini_batches = [10,20]

for batch_size in batch_sizes:
    for epsilon in epsilons:
        for mini_bs in mini_batches:
            for epoch in epochs_to_test:
                # Modify any additional variables here (or put them into their own loop like above)
                args = Arguments(learning_rate=1e-3,
                                epochs=epoch,
                                batch_size=batch_size,
                                epsilon=epsilon,
                                grad_norm=0.5,
                                mode='ghost_mixed',
                                model='beit_base_patch16_224.in22k_ft_in22k',
                                mini_batch_size=mini_bs,
                                pretrained=0)
                epochs, trainf, testf, save_modelf = prepare(args)
                print(f"Running experiment with: {args}")
                train_loss, train_accuracy, test_loss, test_accuracy = main(epochs, trainf, testf, args)
                results.append([epsilon, 0.5, mini_bs, train_loss, train_accuracy, test_loss, test_accuracy])
            with open(f"ham_results/results_no-pretrained_{args.bs}_{args.eps}_{args.mini_bs}.json", 'w') as result_file:
                    result_file.write(json.dumps(results))

==> Building model.. beit_base_patch16_224.in22k_ft_in22k   mode  ghost_mixed
number of parameters:  85767367
Running experiment with: Arguments(learning_rate=0.001, epochs=5, batch_size=800, epsilon=0.5, grad_norm=0.5, mode='ghost_mixed', model='beit_base_patch16_224.in22k_ft_in22k', mini_batch_size=10, pretrained=0)

Epoch: 0


100%|██████████| 3597/3597 [08:51<00:00,  6.77it/s]


0 3597 Loss: 1.933 | Acc: 18.008% (6477/35967)


100%|██████████| 598/598 [00:22<00:00, 26.39it/s]


0 598 Loss: 1.922 | Acc: 22.536% (1347/5977)

Epoch: 1


...

3 299 Loss: 1.836 | Acc: 24.226% (1448/5977)

Epoch: 4


100%|██████████| 1799/1799 [08:20<00:00,  3.60it/s]


4 1799 Loss: 1.851 | Acc: 25.407% (9138/35967)


 51%|█████     | 152/299 [00:12<00:11, 12.51it/s]


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [27]:
# class Arguments:
#     def __init__(self, learning_rate, epochs, batch_size, epsilon, grad_norm, mode, model, mini_batch_size, pretrained):
#         self.lr = learning_rate
#         self.epochs = epochs
#         self.bs = batch_size
#         self.eps = epsilon
#         self.grad_norm = grad_norm
#         self.mode = mode
#         self.model = model
#         self.mini_bs = mini_batch_size
#         self.pretrained = pretrained
#     def __str__(self):
#         return f"Arguments(learning_rate={self.lr}, epochs={self.epochs}, batch_size={self.bs}, epsilon={self.eps}, grad_norm={self.grad_norm}, mode='{self.mode}', model='{self.model}', mini_batch_size={self.mini_bs}, pretrained={self.pretrained})"

# results = [['Epsilon', 'Grad Norm', 'Mini Batch Size', 'Train Loss', 'Train Accuracy', 'Test Loss', 'Test Accuracy']]
# epsilons = [0.5, 1, 2]
# batch_sizes = [800, 1600]
# epochs_to_test = [5, 10]
# mini_batches = [10,20]
# for batch_size in batch_sizes:
#     for epsilon in epsilons:
#         for mini_bs in mini_batches:
#             for epoch in epochs_to_test:
#                 # Modify any additional variables here (or put them into their own loop like above)
#                 args = Arguments(learning_rate=2e-3,
#                                 epochs=epoch,
#                                 batch_size=batch_size,
#                                 epsilon=epsilon,
#                                 grad_norm=0.5,
#                                 mode='ghost_mixed',
#                                 model='beit_base_patch16_224.in22k_ft_in22k',
#                                 mini_batch_size=mini_bs,
#                                 pretrained=1)
#                 epochs, trainf, testf, save_modelf = prepare(args)
#                 print(f"Running experiment with: {args}")
#                 train_loss, train_accuracy, test_loss, test_accuracy = main(epochs, trainf, testf, args)
#                 results.append([epsilon, grad_norm, mini_bs, train_loss, train_accuracy, test_loss, test_accuracy])
#             with open(f"ham_results/results_{args.bs}_{args.eps}_{args.grad_norm}.json", 'w') as result_file:
#                     result_file.write(json.dumps(results))

==> Building model.. beit_base_patch16_224.in22k_ft_in22k   mode  ghost_mixed
number of parameters:  85767367
Running experiment with: Arguments(learning_rate=0.002, epochs=10, batch_size=500, epsilon=0.5, grad_norm=0.1, mode='ghost_mixed', model='beit_base_patch16_224.in22k_ft_in22k', mini_batch_size=100, pretrained=1)

Epoch: 0


100%|██████████| 3597/3597 [08:21<00:00,  7.17it/s]


0 3597 Loss: 1.916 | Acc: 20.814% (7486/35967)


100%|██████████| 598/598 [00:21<00:00, 27.27it/s]


0 598 Loss: 1.873 | Acc: 22.352% (1336/5977)

Epoch: 1


...

9 3597 Loss: 1.930 | Acc: 21.762% (7827/35967)


100%|██████████| 598/598 [00:21<00:00, 27.50it/s]


9 598 Loss: 1.931 | Acc: 24.126% (1442/5977)
==> Building model.. beit_base_patch16_224.in22k_ft_in22k   mode  ghost_mixed
number of parameters:  85767367
Running experiment with: Arguments(learning_rate=0.002, epochs=10, batch_size=500, epsilon=0.5, grad_norm=1, mode='ghost_mixed', model='beit_base_patch16_224.in22k_ft_in22k', mini_batch_size=500, pretrained=1)

Epoch: 0


 49%|████▉     | 1761/3597 [04:30<04:41,  6.51it/s]


KeyboardInterrupt: 

In [24]:

with open(f"ham_results/results_{args.bs}_{args.eps}_{args.grad_norm}.json", 'w') as result_file:
                    result_file.write(json.dumps(results))